## Generate inundation maps for various probabilities
Given a set of probabilities and events, this notebook computes the corresponding inundation maps and outputs them to text files in a format javascript can use.

This script recycles lots of code from [Make_Hazard_Curves_and_Maps.ipynb](Make_Hazard_Curves_and_Maps.ipynb).

### Import libraries needed to run notebook

In [1]:
%pylab inline
from __future__ import print_function
from ptha_paths import data_dir, events_dir, js_data_dir
import sys, os

Populating the interactive namespace from numpy and matplotlib
Assuming that top level of this repository is at: /media/brian/TBdrive/Documents/Coursework/CSE512/Homework/project/tsunami-inundation/ptha_tutorial
    Python codes can be found in codes_dir = /media/brian/TBdrive/Documents/Coursework/CSE512/Homework/project/tsunami-inundation/ptha_tutorial/PythonCode
    Data files can be found in data_dir = /media/brian/TBdrive/Documents/Coursework/CSE512/Homework/project/tsunami-inundation/ptha_tutorial/DataFiles
    Results for each event can be found in events_dir = /media/brian/TBdrive/Documents/Coursework/CSE512/Homework/project/tsunami-inundation/ptha_tutorial/DataFiles/Events
Assuming that top level of the CSE512 repository is at: /media/brian/TBdrive/Documents/Coursework/CSE512/Homework/project/tsunami-inundation
    JavaScript data files can be found in data_dir = /media/brian/TBdrive/Documents/Coursework/CSE512/Homework/project/tsunami-inundation/docs/data


### Set parameters

In [16]:
# The probabilities for which we want to generate inundation maps
probabilities = [1./50, 1./100, 1./200, 1./500, 1./1000, 1./5000, 1./10000]

# A dictionary of sets of events to consider
meta_events         = {}
meta_events['all']  = ['AASZa', 'AASZb', 'AASZc', 'AASZd', 'CSZa', 'CSZb', 'CSZc', 'CSZd', 'CSZe', \
                       'CSZf', 'KmSZa', 'KrSZa', 'SChSZa', 'TOHa']
meta_events['near_field'] = ['CSZa', 'CSZb', 'CSZc', 'CSZd', 'CSZe', 'CSZf']
meta_events['far_field']  = ['AASZa', 'AASZb', 'AASZc', 'AASZd', 'KmSZa', 'KrSZa', 'SChSZa', 'TOHa']

# Specify the range of exceedance values in which we are interested (in meters).
# This determines the resolution and range of values in the inundation maps: 
# their values will be in the set linspace(zeta_a,zeta_b,n_zeta).
zeta_a = 0
zeta_b = 12
n_zeta = 121
zeta   = linspace(zeta_a,zeta_b,n_zeta)

# Resolution of the data
nx = 250
ny = 250

### Some information on the events
The naming convention is the name of a subduction zone, followed by an event number, followed by a realization number, but for our purposes we will think of these as independent events.   The subduction zones represented are:
- AASZ: Alaska-Aleutian
- CSZ: Cascadia
- KmSZ: Kamchatka
- KrSZ: Kurils
- SChSZ: Southern Chile
- TOH: Japan / Tohoku

The inundation patterns shown above were computed as part of a project to develop new techniques for probabilistic tsunami hazard assessment (PTHA), and are reprinted from the report 

- *Probabilistic Tsunami Hazard Assessment (PTHA) for Crescent City, CA.*, by Frank I. Gonzalez, Randall J. LeVeque, Loyce M. Adams, Chris Goldfinger, George R. Priest, and Kelin Wang, <http://hdl.handle.net/1773/25916>

They were computed by running the [GeoClaw](http://www.geoclaw.org) tsunami model with the given earthquake source and then monitoring the maximum inundation depth observed at each point on shore over the course of the entire simulation.  (Actually on a fine grid of points covering the region shown above, with horizontal resolution of 1/3 arcsecond, about 10 meters in latitude and 7 meters in longitude).

### Set the annual probability for each event
We create a dictionary storing the probabilities of each event. These were given to us from Randy. We did not come up with them ourselves.

In [3]:
event_prob           = {}
event_prob['AASZa']  = 1./394.
event_prob['AASZb']  = 1./750.
event_prob['AASZc']  = 1./563.
event_prob['AASZd']  = 1./324.
event_prob['CSZa']   = 1./250. * .0125
event_prob['CSZb']   = 1./250. * .0125
event_prob['CSZc']   = 1./250. * .0750
event_prob['CSZd']   = 1./250. * .5000
event_prob['CSZe']   = 1./250. * .1750
event_prob['CSZf']   = 1./250. * .2250
event_prob['KmSZa']  = 1./50.
event_prob['KrSZa']  = 1./167.
event_prob['SChSZa'] = 1./300.
event_prob['TOHa']   = 1./103.

### Define a function to combine two events

In [4]:
def combine_prob(p1,p2):
    """Returns the probability that event 1 or 2 happens"""
    return 1. - (1 - p1) * (1 - p2)

### Create a function that computes inundation for a given probability
A more commonly used map is obtained by fixing a probability (e.g. $p = 0.01$ for a "100-year" flood map) and plotting the maximum depth expected with this annual probability.   

This requires determining, for each grid point `(i,j)`, the largest value of `k` for which `exceed_prob[k]`  $\geq p$. Then the value `zeta[k]` is the largest exceedance value for which the probability is at least $p$.

Recall that `zeta` is defined to be maximum depth of inundation on shore, or maximum height above MHW offshore.

In [26]:
def compute_zeta(p,exceed_prob):

    # create boolean array K with K[i,j,k] == True only where exceed_prob[i,j,k] > p:
    K = exceed_prob > p

    K[:,:,0] = True
    zeta_p = zeros((nx,ny))
    for i in xrange(nx):
        for j in xrange(ny):
            zeta_p[i,j] = zeta[K[i,j,:]][-1]
    return zeta_p

### Form and save inundation maps

In [28]:
# Loop over the meta events
for meta_event, events in meta_events.iteritems():
    
    # Compute the combined probability of exceeding each exceedance value
    exceed_prob = zeros((nx,ny,n_zeta))
    
    # Loop over all events and update exceed_prob at each grid point by combining
    # current value with the probability Pk of this event:

    for event in events:
        event_dir = os.path.join(events_dir, event)
        hmax_file = os.path.join(event_dir, 'h_eta_small.npy')
        hmax      = load(hmax_file)
        Hmax      = hmax.reshape((nx,ny),order='F')
        for k in xrange(n_zeta):
            Pk = exceed_prob[:,:,k]  # probabilities at all points for one exceedance value zeta_k
            exceed_prob[:,:,k] = where(Hmax > zeta[k], combine_prob(event_prob[event],Pk), Pk)
    
    # Get the inundation array associated with each probability and write to a text file
    for probability in probabilities:
        # Get the inundation array
        inundation_array = compute_zeta(probability,exceed_prob)
        
        # Write to text file
        file_identifier = "%s_eta_pInv%d" %(meta_event, int(1./probability))
        out_file = os.path.join(js_data_dir, "%s.js" % file_identifier  )
        
        header = "var %s = {\"width\":%d,\"height\":%d,\"values\":[\n" %(file_identifier, nx, ny)
        footer = "]};"
        with open(out_file,'w') as f:
            f.write(header)
            first_flag = True
            for el in inundation_array.ravel():
                if first_flag:
                    f.write(str(el))
                    first_flag = False
                else:
                    f.write(',' + str(el))
            f.write(footer)
            
    
        
    print("Done computing inundation map for %s events." % meta_event)

Done computing inundation map for near_field events.
Done computing inundation map for all events.
Done computing inundation map for far_field events.
